## 그래프 요약[함수]

> [함수설명] onepop_mean_plt(막대개수,데이터프레임)

> 막대개수는 히스토그램 빈 개수, 나무상자 그림 출력

In [0]:
def onepop_mean_plt(bin,df): # bin = 막대개수 ,df = 분석할 데이터 <히스토그램, 나무상자그림>
  import matplotlib.pyplot as plt
  plt.subplot(1,2,1)
  n, bins, patches = plt.hist(df,bin, density=True, facecolor='orange', alpha=0.75)
  plt.axvline(x=df.mean(),color='blue',linestyle='--',lw=0.7)
  plt.axvline(x=df.quantile(0.5),color='red',linestyle='--',lw=0.7)
  plt.title('Histogram (mean=%.2f, sd=%.3f)' % (df.mean(),df.std()))
  plt.grid(True)
  plt.subplot(1,2,2)
  plt.title('boxplot')
  plt.boxplot(df,notch=True)
  plt.show()

## 숫자요약 [함수]
>[함수설명] onepop_mean_stat(데이터시리즈)

>> 표본크기, 평균, 표준편차
>> 순서통계량 최소값, 제1사분위, 중앙값, 제3사분위, 최대값, 범위, 사분위범위

In [0]:
def onepop_mean_stat(df):
  res=df.describe()
  print('*** Sample size=%4d Mean=%.3f SD=%.4f \n*** 최소값=%.3f 1사분위=%.3f 중앙값=%.3f 제3사분위=%.3f 최대값=%.3f 범위=%.3f 사분위범위(IQR)=%.3f' 
        %(res[0],res[1],res[2],res[3],res[4],res[5],res[6],res[7],res[7]-res[3],res[6]-res[4]))

## (1-alpha)*100% 신뢰구간 구하기[함수]

> xbar +- t(자유도=n-1,(1-alpha))*sd/sqrt(n), xbar=표본평균, sd=표본표준편차, n=표본크기

> 함수 t.ppf(1-alpha,df=n-1)

[함수설명] onepop_mean_ci(유의수준,데이터시리즈)

In [0]:
def onepop_mean_ci(alpha,df): # alpha = 신뢰구간
  from scipy.stats import t
  import numpy as np
  res=df.describe()
  lb=res[1]-t.ppf(1-alpha/2,df=res[0]-1)*res[2]/np.sqrt(res[0])
  ub=res[1]+t.ppf(1-alpha/2,df=res[0]-1)*res[2]/np.sqrt(res[0])
  print('모평균 추정치=%.3f, 모평균 %2d%% 신뢰구간 : 하한=%.3f, 상한=%.3f' % (res[1],(1-alpha)*100,lb,ub))

[함수설명] 통계량이 주어진 경우 onepop_mean_ci0(유의수준,n,xbar,sd)

In [0]:

def onepop_mean_ci0(alpha,n,xbar,sd): # alpha = 신뢰구간, 주어진 통계량
  from scipy.stats import t
  import numpy as np
  lb=xbar-t.ppf(1-alpha/2,df=n-1)*sd/np.sqrt(n)
  ub=xbar+t.ppf(1-alpha/2,df=n-1)*sd/np.sqrt(n)
  print('모평균 추정치=%.3f, 모평균 %2d%% 신뢰구간 : 하한=%.3f, 상한=%.3f' % (xbar,(1-alpha)*100,lb,ub))

## 가설검정[함수]

> 귀무가설 : 모집단 평균은 mu0이다. mu=mu0 

> 대립가설 : 모집단 평균은 mu0보다 크다. mu!=mu0 (양측가설)

>> mu>mu0, mu<mu0 (단측가설)

> 검정통계량 : (xbar - mu0)/(sd/sqrt(n)) ~ t(자유도=n-1), xbar=표본평균, sd=표본표준편차, n=표본크기

In [0]:
def onepop_mean_ht(alpha,mu0,side,df):
  from scipy.stats import t
  import numpy as np
  res=df.describe()
  ts=(res[1]-mu0)/(res[2]/np.sqrt(res[0]))
  if side==2:
    p=(1-t.cdf(abs(ts),res[0]-1))*2
  else:
    p=1-t.cdf(abs(ts),res[0]-1)
  if p<alpha:
    print('hat(u)=%.3f sd=%.4f test statistics=%.3f pvalues=%.4f (Reject H0)' % (res[1],res[2],ts, p))
  else:
    print('hat(u)=%.3f sd=%.4f test statistics=%.3f pvalues=%.4f (Accept H0)' % (res[1],res[2],ts, p))

[통계량이 주어진 경우]

In [0]:
def onepop_mean_ht0(alpha,mu0,side,n,xbar,sd):
  from scipy.stats import t
  import numpy as np
  ts=(xbar-mu0)/(sd/np.sqrt(n))
  if side==2:
    p=(1-t.cdf(abs(ts),n-1))*2
  else:
    p=1-t.cdf(abs(ts),n-1)
  if p<alpha:
    print('hat(u)=%.3f sd=%.4f test statistics=%.3f pvalues=%.4f (Reject H0)' % (xbar,sd,ts, p))
  else:
    print('hat(u)=%.3f sd=%.4f test statistics=%.3f pvalues=%.4f (Accept H0)' % (xbar,sd,ts, p))

## 이상치 진단 및 삭제[함수]
> [함수설명] extrem(데이터시리즈)

> 출력 결과 : 상한, 하한 극단값 출력

> DF=extrem() -> DF에는 극단치 제거된 데이터시리즈 있음
 

In [0]:
def extrem(df):
  Q1=df.quantile(.25)
  Q3=df.quantile(.75)
  out_up=df[df>Q3+1.5*(Q3-Q1)]
  out_dn=df[df<Q1-1.5*(Q3-Q1)]
  print('상한 극단치 \n',out_up,'\n 하한 극단치 \n',out_dn)
  return df[(df>Q1-1.5*(Q3-Q1))&(df<Q3+1.5*(Q3-Q1))]



---



## 치우침 진단

> (필요 이유) 표본 크기가 작은 경우 모집단의 분포가 치우침이 큰 경우 - 데이터가 충분히 큰 경우에는 문제 없음

> (진단방법)=정규성 검정 Anderson Darling 검정통계량 / Shapiro Wilks 방법

> 귀무가설 : 표본 데이터는 정규분포를 따른다.

> 대립가설 : 표본데이터는 정규분포를 따르지 않는다.

(치우침 해결방법) (우로 치우침) 제곱근 > 로그변환 > -1/X (좌로 치우침) X^2 > X^3

http://wolfpack.hnu.ac.kr/Stat_Notes/elem_stat/Stat_methods/정규변환.pdf

Tukey Power 변환 : X^lambda/(1-lambda)

Box Cox 변환 : (X^lambda-1)/(lambda)

[함수 설명] chk_norm(데이터시리즈) -> 출력결과 (정규성 만족-통계량, 유의확률)
> 정규성 만족하지 않으면 최적 lambda(lam)와 정규변환 데이터(xt)

In [0]:
def chk_norm(df):
  from scipy.stats import shapiro
  stat, p = shapiro(df)
  if p<0.05:
    print('Statistics=%.3f, p=%.3f (Not Normal)' % (stat, p))
    xt,lbm=stats.boxcox(df) #box-cox transformation / xt transformed data
    print('lambda=%.2f'% lbm)
    return (xt, lbm)
  else:
    print('Statistics=%.3f, p=%.3f (Normal)' % (stat, p))

##문제1
> 1.연구문제를 적으시오.

Grand Auto Corporation은 자동차 배터리를 생산한다. 적어도 회사의 평균은 그것의 최고급 Never Die 배터리가 좋다고 주장한다. 소비자 보호 기관은 이 주장을 확인하기 위해 45개의 그러한 배터리를 테스트했다. 이 45개의 배터리의 수명은 63.4개월이며, 표준 편차는 3개월이다. 이러한 모든 배터리의 평균 수명이 65개월 미만임을 테스트해 보십시오. sig-nificance 레벨이 2.5%라면 당신의 결론은 어떻게 될 것인가? p-값


> 2.주어진 내용

n = 45 xber= 63.4 sd = 3


> 단측가설

귀무가설 : 배터리 수명은 65개월이다. mu0 = 65

대립가설 : 배터리 수명은 65개월보다 크다. mu > 65

유의수준 : 5%

In [9]:
onepop_mean_ht0(0.05,65,1,45,63.4,3)

hat(u)=63.400 sd=3.0000 test statistics=-3.578 pvalues=0.0004 (Reject H0)


> 정리

모평균이 65개월일 때, 표본평균이 65개월이 나올 확률은 0.0004로 매우 작으며 유의수준 0.05보다 작아, 0.0004의 낮은 확률로 관측되었기 때문에 H0 모평균 가설이 잘못 되었을 거라 판단하여 H0기각 하며, 표본평균이 63.4로 측정된 것을 보아 실제평균은 65이하일 것 이다.

##문제2
> 1.연구문제

도시 환경과에서 신문 수거 업체를 활용하고자 한다.
수거 업체는 가구당 하루 2파운드 이상 배출해야 수
거하여 수익이 발생한다고 하였다. 환경과에서 수거
업체의 수익을 보장할 수 있는지 알아보기 위하여
148가구에 대하여 조사한 자료

> 2.주어진 내용

귀무가설 : M0 = 2 파운드이다.

대립가설 : M1 < 2 파운드 미만 배출

유의수준 : 5%

In [10]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df1=pd.read_csv('/content/drive/My Drive/통계적방법19_박초연/newspaper.csv')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
df1.head()

,Newspaper,City/suburb
0,1.5,2
1,1.8,2
2,3.2,2
3,2.3,1
4,1.8,2


In [14]:
onepop_mean_ht(0.05,2,1,df1.Newspaper)

hat(u)=2.180 sd=0.9812 test statistics=2.237 pvalues=0.0134 (Reject H0)


> 정리

모평균이 2파운드일 때, 표본평균이 2.18이 나올 확률은 0.0134로 매우 작으며 유의수준 0.05보다 작아, 0.0134의 낮은 확률로 관측되기 때문에 H0 모평균 가설이 잘못 되었을 거라 판단하여 H0기각 하며, 표본평균이 2.18로 측정된 것을 보아 실제평균은 2이상일 것 이다.

##문제3
> 1.연구문제

작년 조사에서 시리얼 구입자들이 아침식사로 지출하는 비용이 평균적으로 $10.6이었다. 올해 작년과 아침식사 비용으로 지출하는 비용이 어떻게 변화하였는지 유의수준 5%에서 검증하시오.

> 2.주어진 내용

귀무가설 : m0=10.6 이다.

대립가설 : m1>10.6 보다 크다.

유의수준 : 5%


In [15]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df2=pd.read_csv('/content/drive/My Drive/통계적방법19_박초연/cereal.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
df2.head()

,Group,Spend
0,1,14.77
1,2,8.15
2,2,8.00
3,4,9.31
4,2,12.09


In [17]:
onepop_mean_ht(0.05,10.6,2,df2.Spend)

hat(u)=10.807 sd=4.3533 test statistics=1.682 pvalues=0.0928 (Accept H0)


> 정리

모평균이 10.6일 때, 표본평균이 10.807이 나올 확률은 0.0928로 매우 크며 유의수준 0.05보다 커, 0.0928의 높은 확률로 관측되기 때문에 H0 모평균 가설이 맞다고 판단하여 H0 채택하며, 평균지출비용은 작년과 비슷할 것이다.

In [18]:
onepop_mean_ci(0.05,df2.Spend)

모평균 추정치=10.807, 모평균 95% 신뢰구간 : 하한=10.566, 상한=11.049


>95%의 확률로 표본평균은 10.807이다.